In [1]:
import pandas as pd
import numpy as np
import gzip
import csv
import os

# User Monitoring Pipeline:

### Create Tables in the AWS instance

In [2]:
# Use the following SQL code in MySQL Workbench in order to create the following tables in the User_Monitoring database:
# 1) ucs: two fields [uuid, score] --> [user's unique identifier, and their current UCS score]
# 2) task_scores: four fields [ts, quiz_task_uuid, user_uuid, task_score] -->
#                             [time stamp, task identifier, user's unique identifier, task_score]
# 3) datahunt_tracker: two fields [datahunt_id, num_rows_processed] --> 
#                                 [datahunt's unique id, number of rows in the datahunt that we processed already]
"""

CREATE DATABASE User_Monitoring;

USE User_Monitoring;

CREATE TABLE ucs (
	uuid CHAR,
    score DECIMAL(6,5) NOT NULL
);

desc UCS;

SELECT * FROM UCS;

----------------
CREATE TABLE task_scores (
	ts TIMESTAMP,
    quiz_task_uuid INT,
    user_uuid INT,
    task_score DECIMAL(6,5)
);

desc task_scores;

SELECT * FROM task_scores;


----------------
CREATE TABLE datahunt_tracker (
	datahunt_id INT,
    num_rows_processed INT
);

desc datahunt_tracker;

SELECT * FROM datahunt_tracker;


"""

'\n\nCREATE DATABASE User_Monitoring;\n\nUSE User_Monitoring;\n\nCREATE TABLE ucs (\n\tuuid CHAR,\n    score DECIMAL(6,5) NOT NULL\n);\n\ndesc UCS;\n\nSELECT * FROM UCS;\n\n----------------\nCREATE TABLE task_scores (\n\tts TIMESTAMP,\n    quiz_task_uuid INT,\n    user_uuid INT,\n    task_score DECIMAL(6,5)\n);\n\ndesc task_scores;\n\nSELECT * FROM task_scores;\n\n\n----------------\nCREATE TABLE datahunt_tracker (\n\tdatahunt_id INT,\n    num_rows_processed INT\n);\n\ndesc datahunt_tracker;\n\nSELECT * FROM datahunt_tracker;\n\n\n'

### Create handler functions to interact with AWS instance

In [3]:
# Install pymysql in the current Jupyter kernel
import sys
!conda install --yes --prefix {sys.prefix} pymysql
# !pip install -t $PWD pymysql
import pymysql
import time
# 1. Install pymysql to local directory
# pip install -t $PWD pymysql

# 2. Write code, then zip it up

# Lambda Permissions:
# AWSLambdaVPCAccessExecutionRole

# Configuration Files
endpoint = 'user-monitoring-database.crnwwfmibeif.us-west-1.rds.amazonaws.com'
username = 'admin'
password = 'user_monitoring'
database_name = 'User_Monitoring'

# Connection
connection = pymysql.connect(
    host=endpoint, user=username, passwd=password, db=database_name)
cursor = connection.cursor()

def lambda_handler(event=None, context=None):
    insert_into_table(event['table'])

def table_to_df(table):
    cursor = connection.cursor()
    cursor.execute('SELECT * from {}'.format(table))
    rows = cursor.fetchall()
    field_names = [i[0] for i in cursor.description]
    df = pd.DataFrame(columns=field_names)
    for row in rows:
        df.loc[len(df.index)] = row
    return df

def display_table(table):
    df = table_to_df(table)
    print(table)
    display(df)

def table_to_csv(table):
    df = table_to_df(table)
    df.to_csv(f'{table}.csv', index=False)

def insert_into_table(table, df):
    """
    TODO: modify function to be dynamic. take in table name to insert into as well
          as dataframe to add into the selected table. add data validation to make sure
          df is correctly formatted for table
    """
    cursor = connection.cursor()
    mysql_query = None
    if table == "ucs":
        # df columns: 'contributor_uuid, score'
        
        insert_ucs = "INSERT INTO `ucs` (`uuid`, `score`) VALUES (%s, %s)"
        def ucs_query(row):
            data_ucs = (row['contributor_uuid'], row['score'])
            cursor.execute(insert_ucs, data_ucs)
            return row
        mysql_query = ucs_query
        
        # cursor.execute('SELECT * from ucs')
    elif table == "task_scores":
        # df columns: 'quiz_task_uuid, contributor_uuid, score'
        task_scores = table_to_df('task_scores').iloc[:, [1,2,3]]
        merged = pd.merge(df, task_scores, how='inner')
        if len(merged) > 0:
            print('overlap, merged table:')
            display(merged)
            return
        
        insert_task_scores = "INSERT INTO task_scores (ts, quiz_task_uuid, user_uuid, task_score) VALUES (%s, %s, %s, %s)"  
        def task_scores_query(row):
            ts = time.strftime("%Y-%m-%d %H:%M:%S")
            quiz_task_uuid = row['quiz_task_uuid']
            contributor_uuid = row['contributor_uuid']
            score = row['score']
            data_task_scores = (ts, quiz_task_uuid, contributor_uuid, score)
            cursor.execute(insert_task_scores, data_task_scores)
            return row
        mysql_query = task_scores_query
        
        # cursor.execute('SELECT * from task_scores')
    elif table == "datahunt_tracker":
        # df columns: 'datahunt_id, num_rows_processed'
        
        insert_datahunt_tracker = "INSERT INTO datahunt_tracker (datahunt_id, num_rows_processed) VALUES (%s, %s)"
        def datahunt_tracker_query(row):
            datahunt_id = row['datahunt_id']
            num_rows_processed = row['num_rows_processed']
            data_datahunt_tracker = (datahunt_id, num_rows_processed)
            cursor.execute(insert_datahunt_tracker, data_datahunt_tracker)
            return row
        
        # cursor.execute('SELECT * from datahunt_tracker')
    
    # run the appropriate query on each row of the given dataframe
    df = df.apply(mysql_query, axis=1)
    connection.commit()
    display_table(table)

def clear_table(table):
    cursor = connection.cursor()
    if table == 'ucs':
        truncate = "TRUNCATE TABLE `ucs`"
    elif table == 'task_scores':
        truncate = "TRUNCATE TABLE `task_scores`"
    elif table == 'datahunt_tracker':
        truncate = "TRUNCATE TABLE `datahunt_tracker`"
    cursor.execute(truncate)
    display_table(table)
    connection.commit()
    
def create_table(table):
    cursor = connection.cursor()
    if table == 'ucs':
        create = "CREATE TABLE `ucs` (`uuid` TINYTEXT, `score` DECIMAL(6,5) NOT NULL)"
    elif table == 'task_scores':
        create = "CREATE TABLE `task_scores` ( \
                                `ts` TIMESTAMP, \
                                `quiz_task_uuid` INT, \
                                `user_uuid` TINYTEXT, \
                                `task_score` DECIMAL(6,5) \
                                )"
    elif table == 'datahunt_tracker':
        create = "CREATE TABLE `datahunt_tracker` ( \
                                `datahunt_id` TINYTEXT, \
                                `num_rows_processed` INT \
                                )"
        
    cursor.execute(create)
    display_table(table)
    connection.commit()

def remake_table(table):
    cursor = connection.cursor()
    if table == 'ucs':
        drop = "DROP TABLE `ucs`"
        create = "CREATE TABLE `ucs` (`uuid` TINYTEXT, `score` DECIMAL(6,5) NOT NULL)"
    elif table == 'task_scores':
        drop = "DROP TABLE `task_scores`"
        create = "CREATE TABLE `task_scores` ( \
                                `ts` TIMESTAMP, \
                                `quiz_task_uuid` TINYTEXT, \
                                `user_uuid` TINYTEXT, \
                                `task_score` DECIMAL(6,5) \
                                )"
    elif table == 'datahunt_tracker':
        drop = "DROP TABLE `datahunt_tracker`"
        create = "CREATE TABLE `datahunt_tracker` ( \
                                `datahunt_id` TINYTEXT, \
                                `num_rows_processed` INT \
                                )"
        
    cursor.execute(drop)
    cursor.execute(create)
    display_table(table)
    connection.commit()
    
def remake_all_tables():
    remake_table('ucs')
    remake_table('task_scores')
    remake_table('datahunt_tracker')
    connection.commit()


Solving environment: done

# All requested packages already installed.



### Function to load in data from /evidence_eric

In [4]:
def load_data(filename):
    df = pd.read_csv(filename)
    return df

In [23]:
test = load_data('./evidence_eric/Covid_Evidencev1.IAA-2022-04-01T1941-Tags.csv_d3Hj2qg.gz')

In [26]:
test

,article_batch_name,article_number,article_filename,article_sha256,article_text_length,tua_group_uuid,tua_group_name,tua_batch_uuid,tua_batch_name,tua_batch_final,...,tua_uuid,namespace,topic_name,case_number,answer_uuid,extra,highlight_count,start_pos,end_pos,target_text
0,CovidArticles/CoronavirusCanHerdImmunityReally...,100056,CoronavirusCanHerdImmunityReally.txt,09a894f06d1157dd7a22a198c6ef78c2b9f4116266d2b0...,4555,1fdbc90c-3c6f-4aad-841f-d00e706e7d7b,Covid_Evidencev1.IAA,c2fbacef-f01c-4e5f-8726-484f35a9b017,tua_csv/2020/06/Adj_Dep_S_IAA_Covid_Evidencev1...,True,...,7a0244e9-9173-4ac6-a3ee-d877a78545aa,Covid_Evidence2020_03_21,27ebbcb0,0,27ebbcb0-94fa-4477-8644-af3f6c9ea03b,"{""answer_text_list"": [""No"", ""No"", ""No""]}",3,2091,2308,"For COVID-19, the R0 is estimated to be 3.28, ..."
1,CovidArticles/CoronavirusCanHerdImmunityReally...,100056,CoronavirusCanHerdImmunityReally.txt,09a894f06d1157dd7a22a198c6ef78c2b9f4116266d2b0...,4555,1fdbc90c-3c6f-4aad-841f-d00e706e7d7b,Covid_Evidencev1.IAA,c2fbacef-f01c-4e5f-8726-484f35a9b017,tua_csv/2020/06/Adj_Dep_S_IAA_Covid_Evidencev1...,True,...,7a0244e9-9173-4ac6-a3ee-d877a78545aa,Covid_Evidence2020_03_21,27ebbcb0,0,27ebbcb0-94fa-4477-8644-af3f6c9ea03b,"{""answer_text_list"": [""No"", ""No"", ""No""]}",3,2312,2501,Achieving herd immunity would require well ove...
2,CovidArticles/CoronavirusCanHerdImmunityReally...,100056,CoronavirusCanHerdImmunityReally.txt,09a894f06d1157dd7a22a198c6ef78c2b9f4116266d2b0...,4555,1fdbc90c-3c6f-4aad-841f-d00e706e7d7b,Covid_Evidencev1.IAA,c2fbacef-f01c-4e5f-8726-484f35a9b017,tua_csv/2020/06/Adj_Dep_S_IAA_Covid_Evidencev1...,True,...,7a0244e9-9173-4ac6-a3ee-d877a78545aa,Covid_Evidence2020_03_21,27ebbcb0,0,27ebbcb0-94fa-4477-8644-af3f6c9ea03b,"{""answer_text_list"": [""No"", ""No"", ""No""]}",3,4117,4228,This means that even in this unlikely \u201cbe...
3,CovidArticles/CoronavirusCanHerdImmunityReally...,100056,CoronavirusCanHerdImmunityReally.txt,09a894f06d1157dd7a22a198c6ef78c2b9f4116266d2b0...,4555,1fdbc90c-3c6f-4aad-841f-d00e706e7d7b,Covid_Evidencev1.IAA,c2fbacef-f01c-4e5f-8726-484f35a9b017,tua_csv/2020/06/Adj_Dep_S_IAA_Covid_Evidencev1...,True,...,25038cad-b145-453e-a2bb-bac991dfa51e,Covid_Evidence2020_03_21,73d8f381,0,73d8f381-e2b9-41fd-ac96-e17752b6c26a,"{""answer_text_list"": [""5: Middling difficulty""]}",0,0,0,NaN
4,CovidArticles/CoronavirusCanHerdImmunityReally...,100056,CoronavirusCanHerdImmunityReally.txt,09a894f06d1157dd7a22a198c6ef78c2b9f4116266d2b0...,4555,1fdbc90c-3c6f-4aad-841f-d00e706e7d7b,Covid_Evidencev1.IAA,c2fbacef-f01c-4e5f-8726-484f35a9b017,tua_csv/2020/06/Adj_Dep_S_IAA_Covid_Evidencev1...,True,...,5dfc6815-f004-4bde-92dd-8afd90550b4e,Covid_Evidence2020_03_21,8eab0dab,0,8eab0dab-e8d4-41ea-af54-5dea4d1e6964,"{""answer_text_list"": [""Much less likely""]}",0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,CovidArticles/intel-boss-confirms-investigatio...,493804,intel-boss-confirms-investigation-into-whether...,2d01e5b4e1274994af1dc3c4f3f13da2c49c0cfee35f05...,4075,1fdbc90c-3c6f-4aad-841f-d00e706e7d7b,Covid_Evidencev1.IAA,71528663-3075-48e2-b17a-77fef41c5fee,Consensus generated at 2021-04-02T223627,True,...,0499aff0-0ae8-4ead-87a6-3a6e8ae1bfdf,Covid_Evidence2020_03_21,01.09.03,1,27ebbcb0-94fa-4477-8644-af3f6c9ea03b,"{""contrib_count"": 2, ""answer_text_list"": []}",0,0,0,NaN
190,CovidArticles/intel-boss-confirms-investigatio...,493804,intel-boss-confirms-investigation-into-whether...,2d01e5b4e1274994af1dc3c4f3f13da2c49c0cfee35f05...,4075,1fdbc90c-3c6f-4aad-841f-d00e706e7d7b,Covid_Evidencev1.IAA,71528663-3075-48e2-b17a-77fef41c5fee,Consensus generated at 2021-04-02T223627,True,...,b0576240-4817-4bba-912d-90f77a5fa2a5,Covid_Evidence2020_03_21,01.01.03,1,b5c3c5d0-4c59-4e26-8996-97cd64e29b93,"{""contrib_count"": 3, ""answer_text_list"": []}",0,0,0,NaN
191,CovidArticles/cb663d51afdc2801bc009b3ef261829d...,493799,new-york-required-nursing-homes-to-admit-medic...,cb663d51afdc2801bc009b3e

In [17]:
testschema = load_data('./evidence_eric/evidence_eric/45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce1626cf4206e159819c.csv')

In [22]:
testschema

,namespace,schema_sha256,topic_uuid,topic_name,topic_options,question_uuid,question_label,question_text,question_type,question_hint_type,...,alpha_distance,question_options,answer_count,answer_uuid,answer_label,answer_content,answer_next_questions,highlight,require_highlight,answer_options
0,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,32fbc0da-a6c1-418d-a1c3-1a62dc841049,Evidence Specialist,"{""highlight"": true, ""version"": ""4"", ""hint_type...",8fe814cf-0d5c-4291-84c0-7e025657d967,T1.Q1,Is a general or singular causal claim made? Hi...,CHECKBOX,NaN,...,nominal,"{""version"": ""4"", ""alpha_distance"": ""nominal"", ...",3,73d7a14a-9ec6-404c-b2b7-a55508af3b76,T1.Q1.A1,"General Causation (In general, X causes Y.)",T1.Q2,1,1,"{""case_numbers"": true, ""highlight"": true, ""ver..."
1,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,32fbc0da-a6c1-418d-a1c3-1a62dc841049,Evidence Specialist,"{""highlight"": true, ""version"": ""4"", ""hint_type...",8fe814cf-0d5c-4291-84c0-7e025657d967,T1.Q1,Is a general or singular causal claim made? Hi...,CHECKBOX,NaN,...,nominal,"{""version"": ""4"", ""alpha_distance"": ""nominal"", ...",3,f91456b2-b2f6-40aa-a347-a537379edb79,T1.Q1.A2,"Singular Causation (In this case, A caused/is ...",T1.Q2,1,1,"{""case_numbers"": true, ""highlight"": true, ""ver..."
2,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,32fbc0da-a6c1-418d-a1c3-1a62dc841049,Evidence Specialist,"{""highlight"": true, ""version"": ""4"", ""hint_type...",8fe814cf-0d5c-4291-84c0-7e025657d967,T1.Q1,Is a general or singular causal claim made? Hi...,CHECKBOX,NaN,...,nominal,"{""version"": ""4"", ""alpha_distance"": ""nominal"", ...",3,b5c3c5d0-4c59-4e26-8996-97cd64e29b93,T1.Q1.A3,No causal claim made,NaN,0,1,"{""case_numbers"": true, ""highlight"": false, ""ve..."
3,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,32fbc0da-a6c1-418d-a1c3-1a62dc841049,Evidence Specialist,"{""highlight"": true, ""version"": ""4"", ""hint_type...",74d61646-391b-4762-8a98-db254ab6d22f,T1.Q2,What evidence is given for the primary causal ...,CHECKBOX,NaN,...,nominal,"{""version"": ""4"", ""alpha_distance"": ""nominal"", ...",9,5a1fb1f4-d8b7-45c0-bce5-7d4c3b91c55f,T1.Q2.A1,Correlation,T1.Q4,1,1,"{""case_numbers"": false, ""highlight"": true, ""ve..."
4,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,32fbc0da-a6c1-418d-a1c3-1a62dc841049,Evidence Specialist,"{""highlight"": true, ""version"": ""4"", ""hint_type...",74d61646-391b-4762-8a98-db254ab6d22f,T1.Q2,What evidence is given for the primary causal ...,CHECKBOX,NaN,...,nominal,"{""version"": ""4"", ""alpha_distance"": ""nominal"", ...",9,ba2d1638-2509-4ce8-9130-39ea26d1d424,T1.Q2.A2,Cause precedes effect,NaN,1,1,"{""case_numbers"": false, ""highlight"": true, ""ve..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,32fbc0da-a6c1-418d-a1c3-1a62dc841049,Evidence Specialist,"{""highlight"": true, ""version"": ""4"", ""hint_type...",b6aedaf6-39e5-4a2b-b01e-8df580dc86dd,T1.Q14,"How confident are you about your answers, on t...",RADIO,NaN,...,nominal,"{""version"": ""4"", ""alpha_distance"": ""nominal"", ...",10,93ef2cd9-7070-469a-9360-fd73294348ad,T1.Q14.A6,6,NaN,0,1,"{""case_numbers"": true, ""highlight"": false, ""ve..."
66,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,32fbc0da-a6c1-418d-a1c3-1a62dc841049,Evidence Specialist,"{""highlight"": true, ""version"": ""4"", ""hint_type...",b6aedaf6-39e5-4a2b-b01e-8df580dc86dd,T1.Q14,"How confident are you about your answers, on t...",RADIO,NaN,...,nominal,"{""version"": ""4"", ""alpha_distance"": ""nominal"", ...",10,d5cfa621-a240-4ac2-b702-13d7117a947a,T1.Q14.A7,7,NaN,0,1,"{""case_numbers"": true, ""highlight"": false, ""ve..."
67,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e68

### Creating Task Scores
This part of the notebook will include steps 2 and 3 from the User Monitoring Pipeline, which includes identifying the consensus answer from the IAA and Gold Standard data (step 2) and creating the corresponding task scores for users who completed this task (step 3).

### Pulling in all the data for a given annotation module (4/3 demo)

In [5]:
data_directory = './evidence_eric'
directory = os.listdir(data_directory)

module_names = ["Covid_ArgumentRelevance", "Covid_Evidence", "Covid_Language", 
                "Covid_Probability", "Covid_Reasoning", "Covid_SourceRelevance"]

module_filemap = {}

def get_module_files(module_name):
    files = []
    for fname in directory:
        if os.path.isfile(data_directory + os.sep + fname):
            # Full path
            if module_name in fname:
                files.append(data_directory + os.sep + fname)
    return files

for module_name in module_names:
    module_filemap[module_name] = get_module_files(module_name)

In [6]:
module_filemap

{'Covid_ArgumentRelevance': ['./evidence_eric\\Covid_ArgumentRelevancev1.2-2022-04-01T1937-DataHunt.csv.gz',
  './evidence_eric\\Covid_ArgumentRelevancev1.2.IAA-2022-04-01T1941-Tags.csv.gz',
  './evidence_eric\\Covid_ArgumentRelevance_2020_03_20v2-Schema.csv.gz'],
 'Covid_Evidence': ['./evidence_eric\\Covid_Evidence2020_03_21-Schema.csv.gz',
  './evidence_eric\\Covid_Evidencev1-2022-04-01T1937-DataHunt.csv.gz',
  './evidence_eric\\Covid_Evidencev1.IAA-2022-04-01T1941-Tags.csv_d3Hj2qg.gz'],
 'Covid_Language': ['./evidence_eric\\Covid_Languagev1.1-2022-04-01T1938-DataHunt.csv.gz',
  './evidence_eric\\Covid_Languagev1.1-Schema.csv.gz',
  './evidence_eric\\Covid_Languagev1.1.IAA-2022-04-03T0251-NegativeTasks.csv'],
 'Covid_Probability': ['./evidence_eric\\Covid_Probability-Schema.csv.gz',
  './evidence_eric\\Covid_Probabilityv1-2022-04-01T1938-DataHunt.csv.gz',
  './evidence_eric\\Covid_Probabilityv1.IAA-2022-04-01T1942-Tags.csv.gz'],
 'Covid_Reasoning': ['./evidence_eric\\Covid_Reasoning-

### Automate Reading and Formatting Schema

In [7]:
def schema_to_type_and_num(ques, schema_path, config='./evidence_eric/'):
    df = pd.read_csv(schema_path, encoding='utf-8')
    override = pd.read_json(config+'schema_override.txt')
    ques = 'T1.Q' + str(ques)
    qrows = df.loc[df['question_label'] == ques]
    q_uuid = qrows['question_uuid'].iloc[0]
    if len(override[override['question_uuid']==q_uuid])>0:
        qrows = override[override['question_uuid']==q_uuid]
    question_type = qrows['question_type'].iloc[0]
    if question_type == 'CHECKBOX':
        question_type = "checklist"
    else:
        question_type = qrows['alpha_distance'].iloc[0]
    answer_count = qrows['answer_count'].iloc[0]
    return question_type, answer_count

In [8]:
schema_to_type_and_num(1, module_filemap['Covid_ArgumentRelevance'][0])

KeyError: 'question_uuid'

## Hardcoded Evidence Schema
Most of this information will be in some sort of schema file (see file 'Evidence2021_05_19-Schema.csv'), but I'm not sure where the schema file is for this specific set of tasks. Thus, I hard coded it with the schema data from https://github.com/Goodly/PEUserMonitoring/blob/master/task-schema/Evidence.txt. Getting this information with the right schema file should be fairly straightforward.


__TODO__: Get schema file and implement method to retrieve the set of scored questions as well as a nested dictionary represented the question schema.

In [9]:
scored_questions = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}

question_schema = {1:{'type':'select_one_nominal', 'num_choices':3},
           2:{'type':'select_all', 'num_choices':9},
           3:{'type':'select_one_nominal', 'num_choices':1},
           4:{'type':'select_one_ordinal', 'num_choices':6},
           5:{'type':'select_one_nominal', 'num_choices':5},
           6:{'type':'select_one_nominal', 'num_choices':3},
           7:{'type':'select_one_ordinal', 'num_choices':1},
           8:{'type':'select_one_ordinal', 'num_choices':5},
           9:{'type':'select_one_ordinal', 'num_choices':3},
           10:{'type':'select_one_ordinal', 'num_choices':5},
           11:{'type':'select_one_ordinal', 'num_choices':5},
           12:{'type':'select_one_ordinal', 'num_choices':4},
           13:{'type':'select_one_ordinal', 'num_choices':10},
           14:{'type':'select_one_ordinal', 'num_choices':10}}

## Hardcoded Covid Sources Schema

In [10]:
scored_questions_covid_sources = {1,2,3,4,5,6,7,8}
question_schema_covid_sources = {
    1:{'type':'select_all', 'num_choices':2},
    2:{'type':'select_one_ordinal', 'num_choices':8},
    3:{'type':'select_one_nominal', 'num_choices':2},
    4:{'type':'select_all', 'num_choices':9},
    5:{'type':'select_all', 'num_choices':9},
    6:{'type':'select_all', 'num_choices':2},
    7:{'type':'select_all', 'num_choices':6},
    8:{'type':'select_one_ordinal', 'num_choices':7},
}

In [11]:
iaa_covid_reasoning = load_data('./evidence_eric/Covid_Reasoningv1.IAA-2022-04-01T1942-Tags.csv.gz')

In [12]:
iaa_covid_reasoning.head()

,article_batch_name,article_number,article_filename,article_sha256,article_text_length,tua_group_uuid,tua_group_name,tua_batch_uuid,tua_batch_name,tua_batch_final,...,tua_uuid,namespace,topic_name,case_number,answer_uuid,extra,highlight_count,start_pos,end_pos,target_text
0,CovidArticles/USMilitaryMayHaveBrought.txt,100055,USMilitaryMayHaveBrought.txt,3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4...,3598,a6a66579-e4e2-44c6-93a9-cde5438e5a7e,Covid_Reasoningv1.IAA,a5176e69-1a32-481d-9d00-4e5e25edc72a,tua_csv/2020/06/Adj_Dep_S_IAA_Covid_Reasoningv...,True,...,cb90d63e-a89b-4869-8002-648b523caad5,Covid_Reasoning,1237866c,0,1237866c-7089-42d1-a1e2-9f90b9d61148,"{""answer_text_list"": [""Misleading reasoning""]}",0,0,0,NaN
1,CovidArticles/USMilitaryMayHaveBrought.txt,100055,USMilitaryMayHaveBrought.txt,3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4...,3598,a6a66579-e4e2-44c6-93a9-cde5438e5a7e,Covid_Reasoningv1.IAA,a5176e69-1a32-481d-9d00-4e5e25edc72a,tua_csv/2020/06/Adj_Dep_S_IAA_Covid_Reasoningv...,True,...,a8d232a2-31cc-4fb5-b0d6-4ae9527916ff,Covid_Reasoning,59994ba5,0,59994ba5-740d-46f2-855a-25e103a1e0d0,"{""answer_text_list"": [""Begging the Question""]}",0,0,0,NaN
2,CovidArticles/USMilitaryMayHaveBrought.txt,100055,USMilitaryMayHaveBrought.txt,3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4...,3598,a6a66579-e4e2-44c6-93a9-cde5438e5a7e,Covid_Reasoningv1.IAA,a5176e69-1a32-481d-9d00-4e5e25edc72a,tua_csv/2020/06/Adj_Dep_S_IAA_Covid_Reasoningv...,True,...,6f2bdd12-5607-4c85-9ca0-41c1e27ae27e,Covid_Reasoning,7831ea7f,0,7831ea7f-613d-4603-9990-60698db3656d,"{""answer_text_list"": [""Appeal to Ignorance""]}",0,0,0,NaN
3,CovidArticles/USMilitaryMayHaveBrought.txt,100055,USMilitaryMayHaveBrought.txt,3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4...,3598,a6a66579-e4e2-44c6-93a9-cde5438e5a7e,Covid_Reasoningv1.IAA,a5176e69-1a32-481d-9d00-4e5e25edc72a,tua_csv/2020/06/Adj_Dep_S_IAA_Covid_Reasoningv...,True,...,48287251-2f3b-45aa-aff1-fb9a42c9635f,Covid_Reasoning,d1ca4968,0,d1ca4968-9555-464f-b098-db4a2f52b7a8,"{""answer_text_list"": [""9""]}",0,0,0,NaN
4,CovidArticles/USMilitaryMayHaveBrought.txt,100055,USMilitaryMayHaveBrought.txt,3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4...,3598,a6a66579-e4e2-44c6-93a9-cde5438e5a7e,Covid_Reasoningv1.IAA,a5176e69-1a32-481d-9d00-4e5e25edc72a,tua_csv/2020/06/Adj_Dep_S_IAA_Covid_Reasoningv...,True,...,e79914c2-5047-44d2-991a-f4b1eb18d310,Covid_Reasoning,dc1a36c6,0,dc1a36c6-945d-4bc0-8b41-a465b07f6a17,"{""answer_text_list"": [""Before therefore becaus...",0,0,0,NaN


## start refactor here

In [40]:
remake_all_tables()

ucs


,uuid,score


task_scores


,ts,quiz_task_uuid,user_uuid,task_score


datahunt_tracker


,datahunt_id,num_rows_processed


In [41]:
def get_answer(question, answer_source, consensus_answers):
    """
    Take in the question and the answer_source, either IAA or Adjudicated / Gold Standard, and adds the
    converged consensus answer to the consensus_answer answer key. This will be an single
    int for select_one questions, or a list of ints for select_all questions.
    """
    question_type = question_schema[question]['type']
    
    if question_type == 'select_one_nominal' or question_type == 'select_one_ordinal':
        assert len(answer_source[answer_source.question_Number == question].agreed_Answer) == 1
        consensus_answers[question] = answer_source[answer_source.question_Number == question].agreed_Answer.iloc[0]
    elif question_type == 'select_all':
        consensus_answers[question] = list(answer_source[answer_source.question_Number == question].agreed_Answer)
    else:
        raise ValueError('Invalid question type')
    
    return consensus_answers

In [42]:
# ucs function that reads accepts values from a csv and a current user's ucs score
def ucs_update_score(user_id):
    def logistic(x, k, offset):
        return 1 / (1 + np.e**(-k * (x - offset)))   
    
    cursor = connection.cursor()
    task_scores = table_to_df('task_scores')
    task_scores = task_scores[task_scores['user_uuid'] == user_id]['task_score'].astype('float')
    print('task_scores:', task_scores)
    last_task_score = task_scores.iloc[-1]
    
    a = 1000
    num_task_scores = len(task_scores)
    n = min(10, int(np.sqrt(num_task_scores)) + 1)
    ucs = table_to_df('ucs')
    
    if user_id not in ucs['uuid']: # if this is the user's first task
        cur_ucs = 0.5
        var_scores = np.var(task_scores.iloc[-n:])
        c = logistic(var_scores / (np.log(num_task_scores + 1) / (np.log(a))), 10, 0.2)
        new_ucs = cur_ucs * (1 - c) + (c) * last_task_score
        query = "INSERT INTO `ucs` (`uuid`, `score`) VALUES (%s, %s)"
        cursor.execute(query, (user_id, new_ucs))
        connection.commit()
    else:
        print('user_id: ', user_id)
        print(ucs[ucs['uuid'] == user_id]['score'].astype('float'))
        cur_ucs = ucs[ucs['uuid'] == user_id]['score'].astype('float')[0]
        var_scores = np.var(task_scores.iloc[-n:])
        c = logistic(var_scores / (np.log(num_task_scores + 1) / (np.log(a))), 10, 0.2)
        new_ucs = cur_ucs * (1 - c) + (c) * last_task_score
        query = "UPDATE `ucs` SET `score` = %s WHERE `uuid` = %s"
        cursor.execute(query, (new_ucs, user_id))
        connection.commit()
    

# Note consensus answer from file is sometimes a string and not an int

In [43]:
# I think the schema file contains scored questions? Not 100% sure so right now it will be hardcoded.
scored_questions = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}

def score_task(iaa_file, adj_file, schema, dh_file):
    # adj_file takes priority
    if adj_file != None:
        file = adj_file
    else:
        file = iaa_file
    
    # cleaning invalid rows in file
    file = file[file.answer_uuid.str.len() > 3]
    
    # these are the only relevant columns for scoring for now, notice highlight data is not included here
    cols = ['answer_uuid', 'question_Number', 'agreed_Answer']
    
    # getting rid of some rows where the above columns were the same, this may represent different 
    # highlights for the same question and answer?
    file = file[cols].drop_duplicates()
    
    consensus_answers = {}

    # create a set of questions that the IAA data determined converged to a consensus
    file_consensus_questions = set(file.question_Number)

    # uses get_answer function to fill in the consensus_answers answer key
    for question in scored_questions:
        if question in file_consensus_questions:
            consensus_answers = get_answer(question, file, consensus_answers)
        else:
            consensus_answers[question] = -1
            
    # return consensus_answers
    
    # narrow down the datahunt to the relevant columns for scoring, getting rid of some rows
    # where the data for the below columns were the same, this may represent different highlights 
    # for the same question and answer? not certain.
    dh = dh_file[['contributor_uuid', 'question_label', 'answer_label']].drop_duplicates()

    # the question and answer labels in the datahunt are in the form 'T1.QX' and 'T1.QX.AX'
    # the below lines strip down to only question number and answer number
    dh['question_label'] = dh['question_label'].str.split('Q').str[1].astype(int)
    dh['answer_label'] = dh['answer_label'].str.split('A').str[1]
    
    # we want to groupby contributor_uuid and question_label to get all the answers a user
    # selected for a particular question, to account for select_all questions. Now, the
    # granularity of df_grouped will be one row per contributor answering a question.
    dh_grouped = dh.groupby(['contributor_uuid', 'question_label']).agg(list).reset_index()
    
    # we only want to score the rows with scored questions (not survey questions like 13 and 14)
    # so we'll filter those out
    dh_grouped = dh_grouped[dh_grouped.question_label.isin(scored_questions)]
    
    def scoring_select_one_nominal(question, answer):
        """
        Takes in a question and the selected answer, returns a score of 0 if the consensus 
        answer is different, and 1 if the consensus answer is the same.
        """
        consensus_answer = int(consensus_answers[question])
        return int(consensus_answer == answer)
    
    def scoring_select_one_ordinal(question, answer):
        """
        Takes in a question and the selected answer, returns a score between 0 and 1 depending
        on how far off the answer is from the consensus answer.
        """
        consensus_answer = int(consensus_answers[question])
        num_choices = question_schema[question]['num_choices']
        return 1 - (abs(answer - consensus_answer) / num_choices)
    
    def scoring_select_all(question, answer_list):
        """
        Takes in a question and the selected answer, returns a score between 0 and 1 depending
        on the accuracy ((True Positive + True Negative) / Total) of the answer selections
        compared to the consensus answer selections.
        """
        answer_set = set(answer_list)
        consensus_answer_set = set(consensus_answers[question])
        num_choices = question_schema[question]['num_choices']

        total_correct = 0
        for answer in range(1, num_choices+1):
            if (answer in answer_set) and (answer in consensus_answer_set):
                total_correct += 1
            elif (answer not in answer_set) and (answer not in consensus_answer_set):
                total_correct += 1
            else:
                total_correct += 0

        return total_correct / num_choices
    
    def scoring(row):
        """
        This is a Pandas apply function, to be applied on axis=1 (on each row).
        Makes a call to one of scoring_select_one_nominal, scoring_select_one_ordinal, and
        scoring_select_all depending on the type of question, returns the outputted score.

        An important note is that right now if neither IAA nor Gold Standard have a consensus
        answer for a question, the consensus_answers answer key will contain a -1 for that 
        question. I currently assume this question should not have been answered due to it
        being a child-question from an incorrectly answered parent question, so I score it
        """
        question = int(row['question_label'])
        answer_list = [int(i) for i in row['answer_label']]

        if consensus_answers[question] == -1:
            return 0

        question_type = question_schema[question]['type']
        if question_type == 'select_one_nominal':
            return scoring_select_one_nominal(question, answer_list[0])
        elif question_type == 'select_one_ordinal':
            return scoring_select_one_ordinal(question, answer_list[0])
        elif question_type == 'select_all':
            return scoring_select_all(question, answer_list)
        else:
            raise ValueError('Invalid question type')
    
    # using the scoring function defined above, we'll create a new column containing the scores
    # for each contributor answering a question.
    dh_grouped['score'] = dh_grouped.apply(scoring, axis=1)
    
    # lastly, we want to get the average score for all task responses, this will be their
    # task score. this is done by a simple groupby on contributor_uuid and mean() aggregate function
    calculated_task_scores = dh_grouped[['contributor_uuid', 'score']].groupby('contributor_uuid').mean().reset_index()
    
    quiz_task_uuid = dh_file["quiz_task_uuid"][0]
    rows_processed = len(dh_file)
    
    calculated_task_scores['quiz_task_uuid'] = quiz_task_uuid
    calculated_task_scores = calculated_task_scores[['quiz_task_uuid', 'contributor_uuid', 'score']]
    
    insert_into_table('task_scores', calculated_task_scores)
    
    for user_id in calculated_task_scores['contributor_uuid']:
        ucs_update_score(user_id)
        
    

In [44]:
# def insert_and_update(calculated_task_scores):
#     insert_into_table('task_scores', calculated_task_scores)
    
#     for user_id in calculated_task_scores['contributor_uuid']:
#         ucs_update_score(user_id)

In [45]:
iaa = pd.read_csv('evidence_eric/evidence_eric/Covid_Evidencev1.IAA-edb1510f-1923-4d6f-a678-95f53d752bea-Tags.csv')
df_full = pd.read_csv('evidence_eric/evidence_eric/Covid_Evidencev1-Task-2224-DataHunt.csv')
score_task(iaa, None, None, df_full)
# insert_and_update(scores)

task_scores


,ts,quiz_task_uuid,user_uuid,task_score
0,2022-04-07 12:51:30,edb1510f-1923-4d6f-a678-95f53d752bea,00f548b7-6b63-4b47-828e-8e416b6ca0e2,0.68241
1,2022-04-07 12:51:30,edb1510f-1923-4d6f-a678-95f53d752bea,070268de-067c-463b-9ad3-5c88292d881e,0.61278
2,2022-04-07 12:51:30,edb1510f-1923-4d6f-a678-95f53d752bea,082a8363-a579-41b4-8918-c166fec3a3a4,0.63000
3,2022-04-07 12:51:30,edb1510f-1923-4d6f-a678-95f53d752bea,09df3ada-e5a8-4419-b78a-e0d1e9b37484,0.47222
4,2022-04-07 12:51:30,edb1510f-1923-4d6f-a678-95f53d752bea,09f279ac-1c34-4a84-8972-3d92b93605a7,0.55556
5,2022-04-07 12:51:30,edb1510f-1923-4d6f-a678-95f53d752bea,0c22ce7c-4641-4bb1-97f4-7a7355f70f25,0.79506
6,2022-04-07 12:51:30,edb1510f-1923-4d6f-a678-95f53d752bea,0e51ab2d-1a03-4d18-be33-fd21a829d19b,0.72593
7,2022-04-07 12:51:30,edb1510f-1923-4d6f-a678-95f53d752bea,1b69eee8-ab95-49dd-8979-9fff7655964d,0.47222
8,2022-04-07 12:51:30,edb1510f-1923-4d6f-a678-95f53d752bea,21ffd986-c219-43a0-b82f-4cc460da628d,0.81481
9,2022-04-07 12:51:30,edb1510f-1923-4d6f-a678-95f53d752bea,24640f45-b90b-40dc-a848-9e03fdfbbf91,0.47857


task_scores: 0    0.68241
Name: task_score, dtype: float64
task_scores: 1    0.61278
Name: task_score, dtype: float64
task_scores: 2    0.63
Name: task_score, dtype: float64
task_scores: 3    0.47222
Name: task_score, dtype: float64
task_scores: 4    0.55556
Name: task_score, dtype: float64
task_scores: 5    0.79506
Name: task_score, dtype: float64
task_scores: 6    0.72593
Name: task_score, dtype: float64
task_scores: 7    0.47222
Name: task_score, dtype: float64
task_scores: 8    0.81481
Name: task_score, dtype: float64
task_scores: 9    0.47857
Name: task_score, dtype: float64
task_scores: 10    0.74167
Name: task_score, dtype: float64
task_scores: 11    0.55556
Name: task_score, dtype: float64
task_scores: 12    0.60909
Name: task_score, dtype: float64
task_scores: 13    0.67099
Name: task_score, dtype: float64
task_scores: 14    0.63182
Name: task_score, dtype: float64
task_scores: 15    0.72071
Name: task_score, dtype: float64
task_scores: 16    0.55556
Name: task_score, dtype: f

In [37]:
display_table("task_scores")

task_scores


,ts,quiz_task_uuid,user_uuid,task_score
0,2022-04-07 12:49:53,edb1510f-1923-4d6f-a678-95f53d752bea,00f548b7-6b63-4b47-828e-8e416b6ca0e2,0.68241
1,2022-04-07 12:49:53,edb1510f-1923-4d6f-a678-95f53d752bea,070268de-067c-463b-9ad3-5c88292d881e,0.61278
2,2022-04-07 12:49:53,edb1510f-1923-4d6f-a678-95f53d752bea,082a8363-a579-41b4-8918-c166fec3a3a4,0.63000
3,2022-04-07 12:49:53,edb1510f-1923-4d6f-a678-95f53d752bea,09df3ada-e5a8-4419-b78a-e0d1e9b37484,0.47222
4,2022-04-07 12:49:53,edb1510f-1923-4d6f-a678-95f53d752bea,09f279ac-1c34-4a84-8972-3d92b93605a7,0.55556
5,2022-04-07 12:49:53,edb1510f-1923-4d6f-a678-95f53d752bea,0c22ce7c-4641-4bb1-97f4-7a7355f70f25,0.79506
6,2022-04-07 12:49:53,edb1510f-1923-4d6f-a678-95f53d752bea,0e51ab2d-1a03-4d18-be33-fd21a829d19b,0.72593
7,2022-04-07 12:49:53,edb1510f-1923-4d6f-a678-95f53d752bea,1b69eee8-ab95-49dd-8979-9fff7655964d,0.47222
8,2022-04-07 12:49:53,edb1510f-1923-4d6f-a678-95f53d752bea,21ffd986-c219-43a0-b82f-4cc460da628d,0.81481
9,2022-04-07 12:49:53,edb1510f-1923-4d6f-a678-95f53d752bea,24640f45-b90b-40dc-a848-9e03fdfbbf91,0.47857


In [38]:
display_table("ucs")

ucs


,uuid,score
0,00f548b7-6b63-4b47-828e-8e416b6ca0e2,0.52174
1,070268de-067c-463b-9ad3-5c88292d881e,0.51344
2,082a8363-a579-41b4-8918-c166fec3a3a4,0.51550
3,09df3ada-e5a8-4419-b78a-e0d1e9b37484,0.49669
4,09f279ac-1c34-4a84-8972-3d92b93605a7,0.50662
5,0c22ce7c-4641-4bb1-97f4-7a7355f70f25,0.53517
6,0e51ab2d-1a03-4d18-be33-fd21a829d19b,0.52693
7,1b69eee8-ab95-49dd-8979-9fff7655964d,0.49669
8,21ffd986-c219-43a0-b82f-4cc460da628d,0.53753
9,24640f45-b90b-40dc-a848-9e03fdfbbf91,0.49745


In [39]:
remake_all_tables()

ucs


,uuid,score


task_scores


,ts,quiz_task_uuid,user_uuid,task_score


datahunt_tracker


,datahunt_id,num_rows_processed


# end refactor here


## Preprocessing of IAA and Gold Standard Data

In [52]:
# read in the data
adjudicated = pd.read_csv('evidence_eric/evidence_eric/Covid_Evidence2020_03_21.adjudicated-edb1510f-1923-4d6f-a678-95f53d752bea-Tags.csv')
iaa = pd.read_csv('evidence_eric/evidence_eric/Covid_Evidencev1.IAA-edb1510f-1923-4d6f-a678-95f53d752bea-Tags.csv')

# iaa = load_data('./evidence_eric/Covid_SourceRelevancev1.IAA-2022-04-01T1942-Tags.csv_tm1x8SV.gz')
# getting rid of some rows where the answer was invalid, probably represents some other metadata
iaa = iaa[iaa.answer_uuid.str.len() > 3]

In [53]:
iaa.head()

,article_num,article_sha256,article_id,article_filename,source_task_uuid,tua_uuid,namespace,schema_sha256,question_Number,answer_uuid,...,agreed_Answer,coding_perc_agreement,highlighted_indices,agreement_score,num_users,num_answer_choices,target_text,question_text,answer_text,article_text_length
0,100059,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,raw_304b537e0ed21179a29ed28da28057d338e67330ae...,Covid_article_for_PE_S&S&S.txt,edb1510f-1923-4d6f-a678-95f53d752bea,a723537a-f11a-41dd-bf5b-668cef67a5de,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,1,73d7a14a-9ec6-404c-b2b7-a55508af3b76,...,1,0.649123,"[1401, 1402, 1403, 1404, 1405, 1406, 1407, 140...",0.649123,57,3,Many quarantined individuals experienced both ...,Is a general or singular causal claim made? Hi...,"General Causation (In general, X causes Y.)",6794
3,100059,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,raw_304b537e0ed21179a29ed28da28057d338e67330ae...,Covid_article_for_PE_S&S&S.txt,edb1510f-1923-4d6f-a678-95f53d752bea,a723537a-f11a-41dd-bf5b-668cef67a5de,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,2,5a1fb1f4-d8b7-45c0-bce5-7d4c3b91c55f,...,1,0.725000,"[1401, 1402, 1403, 1404, 1405, 1406, 1407, 140...",0.725000,40,9,Many quarantined individuals experienced both ...,What evidence is given for the primary causal ...,Correlation,6794
4,100059,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,raw_304b537e0ed21179a29ed28da28057d338e67330ae...,Covid_article_for_PE_S&S&S.txt,edb1510f-1923-4d6f-a678-95f53d752bea,a723537a-f11a-41dd-bf5b-668cef67a5de,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,2,ba2d1638-2509-4ce8-9130-39ea26d1d424,...,2,0.350000,"[1227, 1228, 1229, 1230, 1231, 1232, 1233, 123...",0.350000,40,9,outcomes of people who were quarantined//break...,What evidence is given for the primary causal ...,Cause precedes effect,6794
5,100059,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,raw_304b537e0ed21179a29ed28da28057d338e67330ae...,Covid_article_for_PE_S&S&S.txt,edb1510f-1923-4d6f-a678-95f53d752bea,a723537a-f11a-41dd-bf5b-668cef67a5de,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,2,fee5e5ec-ce13-43e7-aca6-5babee4eb8a5,...,3,0.675000,"[1177, 1178, 1179, 1180, 1181, 1182, 1183, 118...",0.675000,40,9,evaluated 24 studies//break//during outbreaks ...,What evidence is given for the primary causal ...,The correlation appears across multiple indepe...,6794
7,100059,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,raw_304b537e0ed21179a29ed28da28057d338e67330ae...,Covid_article_for_PE_S&S&S.txt,edb1510f-1923-4d6f-a678-95f53d752bea,a723537a-f11a-41dd-bf5b-668cef67a5de,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,2,780989ca-2765-4989-9cc3-08f3874489e0,...,5,0.550000,"[1187, 1188, 1189, 1190, 1191, 1192, 1193, 119...",0.550000,40,9,24 studies//break//psychological outcomes of p...,What evidence is given for the primary causal ...,An experimental study was conducted (natural e...,6794


In [54]:
# these are the only relevant columns for scoring for now, notice highlight data is not included here
cols = ['answer_uuid', 'question_Number', 'agreed_Answer']

# getting rid of some rows where the above columns were the same, this may represent different 
# highlights for the same question and answer?
adjudicated = adjudicated[cols].drop_duplicates()
iaa = iaa[cols].drop_duplicates()

Below cells just show the format of the preprocessed IAA and Adjudicated / Gold Standard data.

In [55]:
adjudicated.head(3)

,answer_uuid,question_Number,agreed_Answer
0,73d7a14a-9ec6-404c-b2b7-a55508af3b76,1,1
1,5a1fb1f4-d8b7-45c0-bce5-7d4c3b91c55f,2,1
3,ba2d1638-2509-4ce8-9130-39ea26d1d424,2,2


In [56]:
iaa.head(3)

,answer_uuid,question_Number,agreed_Answer
0,73d7a14a-9ec6-404c-b2b7-a55508af3b76,1,1
3,5a1fb1f4-d8b7-45c0-bce5-7d4c3b91c55f,2,1
4,ba2d1638-2509-4ce8-9130-39ea26d1d424,2,2


## Creating The Answer Key

In [57]:
# consensus answer key
consensus_answers = {}

def get_answer(question, answer_source):
    """
    Take in the question and the answer_source, either IAA or Adjudicated / Gold Standard, and adds the
    converged consensus answer to the consensus_answer answer key. This will be an single
    int for select_one questions, or a list of ints for select_all questions.
    """
    question_type = question_schema[question]['type']
    
    if question_type == 'select_one_nominal' or question_type == 'select_one_ordinal':
        assert len(answer_source[answer_source.question_Number == question].agreed_Answer) == 1
        consensus_answers[question] = answer_source[answer_source.question_Number == question].agreed_Answer.iloc[0]
    elif question_type == 'select_all':
        consensus_answers[question] = list(answer_source[answer_source.question_Number == question].agreed_Answer)
    else:
        raise ValueError('Invalid question type')

In [58]:
# create a set of questions that the Adjudicated / Gold Standard data determined converged to a consensus
# adjudicated_consensus_questions = set(adjudicated.question_Number)

# create a set of questions that the IAA data determined converged to a consensus
iaa_consensus_questions = set(iaa.question_Number)

# uses get_answer function to fill in the consensus_answers answer key
for question in scored_questions:
    # if question in adjudicated_consensus_questions:
    #     get_answer(question, adjudicated)
    # elif question in iaa_consensus_questions:
    #     get_answer(question, iaa)
    # else:
    #     consensus_answers[question] = -1
    if question in iaa_consensus_questions:
        get_answer(question, iaa)
    else:
        consensus_answers[question] = -1

What the consensus key looks like:

In [59]:
consensus_answers

{1: '1',
 2: ['1', '2', '3', '5'],
 3: -1,
 4: '4',
 5: '2',
 6: '3',
 7: -1,
 8: '3',
 9: '2',
 10: '3',
 11: '3',
 12: '3'}

## Scoring Users

In [60]:
def scoring_select_one_nominal(question, answer):
    """
    Takes in a question and the selected answer, returns a score of 0 if the consensus 
    answer is different, and 1 if the consensus answer is the same.
    """
    consensus_answer = int(consensus_answers[question])
    return int(consensus_answer == answer)

In [61]:
def scoring_select_one_ordinal(question, answer):
    """
    Takes in a question and the selected answer, returns a score between 0 and 1 depending
    on how far off the answer is from the consensus answer.
    """
    consensus_answer = int(consensus_answers[question])
    num_choices = question_schema[question]['num_choices']
    return 1 - (abs(answer - consensus_answer) / num_choices)

In [62]:
def scoring_select_all(question, answer_list):
    """
    Takes in a question and the selected answer, returns a score between 0 and 1 depending
    on the accuracy ((True Positive + True Negative) / Total) of the answer selections
    compared to the consensus answer selections.
    """
    answer_set = set(answer_list)
    consensus_answer_set = set(consensus_answers[question])
    num_choices = question_schema[question]['num_choices']
    
    total_correct = 0
    for answer in range(1, num_choices+1):
        if (answer in answer_set) and (answer in consensus_answer_set):
            total_correct += 1
        elif (answer not in answer_set) and (answer not in consensus_answer_set):
            total_correct += 1
        else:
            total_correct += 0
        
    return total_correct / num_choices

In [63]:
def scoring(row):
    """
    This is a Pandas apply function, to be applied on axis=1 (on each row).
    Makes a call to one of scoring_select_one_nominal, scoring_select_one_ordinal, and
    scoring_select_all depending on the type of question, returns the outputted score.
    
    An important note is that right now if neither IAA nor Gold Standard have a consensus
    answer for a question, the consensus_answers answer key will contain a -1 for that 
    question. I currently assume this question should not have been answered due to it
    being a child-question from an incorrectly answered parent question, so I score it
    """
    question = int(row['question_label'])
    answer_list = [int(i) for i in row['answer_label']]
    
    if consensus_answers[question] == -1:
        return 0
    
    question_type = question_schema[question]['type']
    if question_type == 'select_one_nominal':
        return scoring_select_one_nominal(question, answer_list[0])
    elif question_type == 'select_one_ordinal':
        return scoring_select_one_ordinal(question, answer_list[0])
    elif question_type == 'select_all':
        return scoring_select_all(question, answer_list)
    else:
        raise ValueError('Invalid question type')

In [64]:
# read in the datahunt
df_full = pd.read_csv('evidence_eric/evidence_eric/Covid_Evidencev1-Task-2224-DataHunt.csv')
# df_full = load_data('./evidence_eric/Covid_SourceRelevancev1-2022-04-01T1939-DataHunt.csv.gz')

In [65]:
df_full

,namespace,schema_sha256,quiz_task_uuid,task_url,tua_uuid,article_batch_name,article_number,article_filename,article_sha256,article_text_length,...,answer_label,answer_content,answer_uuid,submitted_tua_uuid,answer_text,case_number,highlight_count,start_pos,end_pos,target_text
0,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,edb1510f-1923-4d6f-a678-95f53d752bea,https://pe.goodlylabs.org/project/Covid_Eviden...,a723537a-f11a-41dd-bf5b-668cef67a5de,CovidArticles/Covid_article_for_PE_S&S&S.txt,100059,Covid_article_for_PE_S&S&S.txt,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,6794,...,T1.Q1.A1,"General Causation (In general, X causes Y.)",73d7a14a-9ec6-404c-b2b7-a55508af3b76,50a87210-bcda-459b-9be6-5587a1459012,"General Causation (In general, X causes Y.)",1,1,7,57,Social distancing comes with psychological fal...
1,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,edb1510f-1923-4d6f-a678-95f53d752bea,https://pe.goodlylabs.org/project/Covid_Eviden...,a723537a-f11a-41dd-bf5b-668cef67a5de,CovidArticles/Covid_article_for_PE_S&S&S.txt,100059,Covid_article_for_PE_S&S&S.txt,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,6794,...,T1.Q2.A1,Correlation,5a1fb1f4-d8b7-45c0-bce5-7d4c3b91c55f,09412d04-f88d-4328-a8c0-556bdc47d669,Correlation,1,1,1401,1937,Many\xa0quarantined individuals experienced bo...
2,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,edb1510f-1923-4d6f-a678-95f53d752bea,https://pe.goodlylabs.org/project/Covid_Eviden...,a723537a-f11a-41dd-bf5b-668cef67a5de,CovidArticles/Covid_article_for_PE_S&S&S.txt,100059,Covid_article_for_PE_S&S&S.txt,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,6794,...,T1.Q2.A2,Cause precedes effect,ba2d1638-2509-4ce8-9130-39ea26d1d424,b1f0cf0f-a6ae-4fb1-8e6d-551b652c88f4,Cause precedes effect,1,1,1213,1266,psychological outcomes of people who were quar...
3,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,edb1510f-1923-4d6f-a678-95f53d752bea,https://pe.goodlylabs.org/project/Covid_Eviden...,a723537a-f11a-41dd-bf5b-668cef67a5de,CovidArticles/Covid_article_for_PE_S&S&S.txt,100059,Covid_article_for_PE_S&S&S.txt,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,6794,...,T1.Q2.A3,The correlation appears across multiple indepe...,fee5e5ec-ce13-43e7-aca6-5babee4eb8a5,170e0c43-a9f0-447c-aa35-4be7864e1dcf,The correlation appears across multiple indepe...,1,1,1326,1399,"SARS, H1N1 flu, Ebola and other infectious dis..."
4,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,edb1510f-1923-4d6f-a678-95f53d752bea,https://pe.goodlylabs.org/project/Covid_Eviden...,a723537a-f11a-41dd-bf5b-668cef67a5de,CovidArticles/Covid_article_for_PE_S&S&S.txt,100059,Covid_article_for_PE_S&S&S.txt,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,6794,...,T1.Q4.A6,Can't tell; not enough info,7369a857-fd09-4a36-8e26-e64e8d5a6641,44fc7d46-3135-41c9-a2ee-244e76de09af,Can't tell; not enough info,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
768,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,edb1510f-1923-4d6f-a678-95f53d752bea,https://pe.goodlylabs.org/project/Covid_Eviden...,a723537a-f11a-41dd-bf5b-668cef67a5de,CovidArticles/Covid_article_for_PE_S&S&S.txt,100059,Covid_article_for_PE_S&S&S.txt,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,6794,...,T1.Q10.A5,Very Unlikely,443d94fe-c0a2-4a8b-819f-9253a570f788,d0db2afe-8fba-4f2a-b0fc-86a9182a2627,Very Unlikely,0,0,0,0,NaN
769,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,edb1510f-1923-4d6f-a678-95f53d752bea,https://pe.goodlylabs.org/project/Covid_Eviden...,a723537a-f11a-41dd-bf5b-668cef67a5de,CovidArticles/Covid_article_for_PE_S&S&S.txt,100059,Covid_article_for_PE_S&S&S.txt,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,6794,...,T1.Q11.A4,Somewhat Unlikely,6a8b58aa-2748-4697-995b-c1a34149f70d,c934c5a4-eb4d-4fa5-9d7c-411d5d566b14,Somewhat

In [66]:
# narrow down the datahunt to the relevant columns for scoring, getting rid of some rows
# where the data for the below columns were the same, this may represent different highlights 
# for the same question and answer? not certain.
df = df_full[['contributor_uuid', 'question_label', 'answer_label']].drop_duplicates()

# the question and answer labels in the datahunt are in the form 'T1.QX' and 'T1.QX.AX'
# the below lines strip down to only question number and answer number
df['question_label'] = df['question_label'].str.split('Q').str[1].astype(int)
df['answer_label'] = df['answer_label'].str.split('A').str[1]

In [67]:
df

,contributor_uuid,question_label,answer_label
0,e1ae8875-a398-4dde-8f4e-4b21109784e3,1,1
1,e1ae8875-a398-4dde-8f4e-4b21109784e3,2,1
2,e1ae8875-a398-4dde-8f4e-4b21109784e3,2,2
3,e1ae8875-a398-4dde-8f4e-4b21109784e3,2,3
4,e1ae8875-a398-4dde-8f4e-4b21109784e3,4,6
...,...,...,...
768,bd786026-bad5-4fa8-9a3a-38ca03a16412,10,5
769,bd786026-bad5-4fa8-9a3a-38ca03a16412,11,4
770,bd786026-bad5-4fa8-9a3a-38ca03a16412,12,2
771,bd786026-bad5-4fa8-9a3a-38ca03a16412,13,6


In [68]:
# we want to groupby contributor_uuid and question_label to get all the answers a user
# selected for a particular question, to account for select_all questions. Now, the
# granularity of df_grouped will be one row per contributor answering a question.
df_grouped = df.groupby(['contributor_uuid', 'question_label']).agg(list).reset_index()

In [69]:
df_grouped

,contributor_uuid,question_label,answer_label
0,00f548b7-6b63-4b47-828e-8e416b6ca0e2,1,[1]
1,00f548b7-6b63-4b47-828e-8e416b6ca0e2,2,"[3, 5, 8, 4]"
2,00f548b7-6b63-4b47-828e-8e416b6ca0e2,3,[1]
3,00f548b7-6b63-4b47-828e-8e416b6ca0e2,4,[6]
4,00f548b7-6b63-4b47-828e-8e416b6ca0e2,5,[2]
...,...,...,...
549,fd6f6837-7881-4943-8129-b7ea0f0fe1b6,10,[4]
550,fd6f6837-7881-4943-8129-b7ea0f0fe1b6,11,[4]
551,fd6f6837-7881-4943-8129-b7ea0f0fe1b6,12,[2]
552,fd6f6837-7881-4943-8129-b7ea0f0fe1b6,13,[2]


In [70]:
# we only want to score the rows with scored questions (not survey questions like 13 and 14)
# so we'll filter those out
df_grouped = df_grouped[df_grouped.question_label.isin(scored_questions)]

In [71]:
df_grouped

,contributor_uuid,question_label,answer_label
0,00f548b7-6b63-4b47-828e-8e416b6ca0e2,1,[1]
1,00f548b7-6b63-4b47-828e-8e416b6ca0e2,2,"[3, 5, 8, 4]"
2,00f548b7-6b63-4b47-828e-8e416b6ca0e2,3,[1]
3,00f548b7-6b63-4b47-828e-8e416b6ca0e2,4,[6]
4,00f548b7-6b63-4b47-828e-8e416b6ca0e2,5,[2]
...,...,...,...
547,fd6f6837-7881-4943-8129-b7ea0f0fe1b6,8,[5]
548,fd6f6837-7881-4943-8129-b7ea0f0fe1b6,9,[1]
549,fd6f6837-7881-4943-8129-b7ea0f0fe1b6,10,[4]
550,fd6f6837-7881-4943-8129-b7ea0f0fe1b6,11,[4]


In [72]:
# using the scoring function defined above, we'll create a new column containing the scores
# for each contributor answering a question.
df_grouped['score'] = df_grouped.apply(scoring, axis=1)

C:\Users\16spook\AppData\Local\Temp/ipykernel_8200/2111373349.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grouped['score'] = df_grouped.apply(scoring, axis=1)


This is the current format of df_grouped.

In [73]:
df_grouped.head(3)

,contributor_uuid,question_label,answer_label,score
0,00f548b7-6b63-4b47-828e-8e416b6ca0e2,1,[1],1.000000
1,00f548b7-6b63-4b47-828e-8e416b6ca0e2,2,"[3, 5, 8, 4]",0.555556
2,00f548b7-6b63-4b47-828e-8e416b6ca0e2,3,[1],0.000000


In [74]:
# lastly, we want to get the average score for all task responses, this will be their
# task score. this is done by a simple groupby on contributor_uuid and mean() aggregate function
calculated_task_scores = df_grouped[['contributor_uuid', 'score']].groupby('contributor_uuid').mean().reset_index()

In [75]:
quiz_task_uuid = df_full['quiz_task_uuid'][0]
quiz_task_uuid

'edb1510f-1923-4d6f-a678-95f53d752bea'

The below value is the number of rows processed in this datahunt, to be used for updating the datahunt tracking table.

In [76]:
rows_processed = len(df_full)
rows_processed

773

The below value represents an identifier for each datahunt, meaning an identifier for each task since each datahunt corresponds to one task.

This is what the final task_scores output will look like:

In [77]:
calculated_task_scores['quiz_task_uuid'] = quiz_task_uuid
calculated_task_scores = calculated_task_scores[['quiz_task_uuid', 'contributor_uuid', 'score']]
calculated_task_scores

,quiz_task_uuid,contributor_uuid,score
0,edb1510f-1923-4d6f-a678-95f53d752bea,00f548b7-6b63-4b47-828e-8e416b6ca0e2,0.682407
1,edb1510f-1923-4d6f-a678-95f53d752bea,070268de-067c-463b-9ad3-5c88292d881e,0.612778
2,edb1510f-1923-4d6f-a678-95f53d752bea,082a8363-a579-41b4-8918-c166fec3a3a4,0.630000
3,edb1510f-1923-4d6f-a678-95f53d752bea,09df3ada-e5a8-4419-b78a-e0d1e9b37484,0.472222
4,edb1510f-1923-4d6f-a678-95f53d752bea,09f279ac-1c34-4a84-8972-3d92b93605a7,0.555556
5,edb1510f-1923-4d6f-a678-95f53d752bea,0c22ce7c-4641-4bb1-97f4-7a7355f70f25,0.795062
6,edb1510f-1923-4d6f-a678-95f53d752bea,0e51ab2d-1a03-4d18-be33-fd21a829d19b,0.725926
7,edb1510f-1923-4d6f-a678-95f53d752bea,1b69eee8-ab95-49dd-8979-9fff7655964d,0.472222
8,edb1510f-1923-4d6f-a678-95f53d752bea,21ffd986-c219-43a0-b82f-4cc460da628d,0.814815
9,edb1510f-1923-4d6f-a678-95f53d752bea,24640f45-b90b-40dc-a848-9e03fdfbbf91,0.478571


## Updating and Retrieving Data from the User-Monitoring Database

#### Step 1) Insert calculated_task_scores into the task scores table and check that the task_scores table updated properly

In [78]:
insert_into_table('task_scores', calculated_task_scores)

task_scores


,ts,quiz_task_uuid,user_uuid,task_score
0,2022-04-07 12:40:40,edb1510f-1923-4d6f-a678-95f53d752bea,00f548b7-6b63-4b47-828e-8e416b6ca0e2,0.68241
1,2022-04-07 12:40:40,edb1510f-1923-4d6f-a678-95f53d752bea,070268de-067c-463b-9ad3-5c88292d881e,0.61278
2,2022-04-07 12:40:40,edb1510f-1923-4d6f-a678-95f53d752bea,082a8363-a579-41b4-8918-c166fec3a3a4,0.63000
3,2022-04-07 12:40:40,edb1510f-1923-4d6f-a678-95f53d752bea,09df3ada-e5a8-4419-b78a-e0d1e9b37484,0.47222
4,2022-04-07 12:40:40,edb1510f-1923-4d6f-a678-95f53d752bea,09f279ac-1c34-4a84-8972-3d92b93605a7,0.55556
5,2022-04-07 12:40:40,edb1510f-1923-4d6f-a678-95f53d752bea,0c22ce7c-4641-4bb1-97f4-7a7355f70f25,0.79506
6,2022-04-07 12:40:40,edb1510f-1923-4d6f-a678-95f53d752bea,0e51ab2d-1a03-4d18-be33-fd21a829d19b,0.72593
7,2022-04-07 12:40:40,edb1510f-1923-4d6f-a678-95f53d752bea,1b69eee8-ab95-49dd-8979-9fff7655964d,0.47222
8,2022-04-07 12:40:40,edb1510f-1923-4d6f-a678-95f53d752bea,21ffd986-c219-43a0-b82f-4cc460da628d,0.81481
9,2022-04-07 12:40:40,edb1510f-1923-4d6f-a678-95f53d752bea,24640f45-b90b-40dc-a848-9e03fdfbbf91,0.47857


#### Step 2) Update User Credibility Score:
Algorithm: Once per row in calculated_task_scores:
1.  if contributor_uuid not in ucs, initialize user in ucs (insert_to_table(ucs)) with baseline score of 0.5
2. call scoring function: takes in calculated_task_scores, calculates c_value, accesses ucs data
3. update ucs table in place

In [169]:
# ucs function that reads accepts values from a csv and a current user's ucs score
def ucs_update_score(user_id):
    def logistic(x, k, offset):
        return 1 / (1 + np.e**(-k * (x - offset)))   
    
    cursor = connection.cursor()
    task_scores = table_to_df('task_scores')
    task_scores = task_scores[task_scores['user_uuid'] == user_id]['task_score'].astype('float')
    print('task_scores:', task_scores)
    last_task_score = task_scores.iloc[-1]
    
    a = 1000
    num_task_scores = len(task_scores)
    n = min(10, int(np.sqrt(num_task_scores)) + 1)
    ucs = table_to_df('ucs')
    
    if user_id not in ucs['uuid']: # if this is the user's first task
        cur_ucs = 0.5
        var_scores = np.var(task_scores.iloc[-n:])
        c = logistic(var_scores / (np.log(num_task_scores + 1) / (np.log(a))), 10, 0.2)
        new_ucs = cur_ucs * (1 - c) + (c) * last_task_score
        query = "INSERT INTO `ucs` (`uuid`, `score`) VALUES (%s, %s)"
        cursor.execute(query, (user_id, new_ucs))
        connection.commit()
    else:
        print('user_id: ', user_id)
        print(ucs[ucs['uuid'] == user_id]['score'].astype('float'))
        cur_ucs = ucs[ucs['uuid'] == user_id]['score'].astype('float')[0]
        var_scores = np.var(task_scores.iloc[-n:])
        c = logistic(var_scores / (np.log(num_task_scores + 1) / (np.log(a))), 10, 0.2)
        new_ucs = cur_ucs * (1 - c) + (c) * last_task_score
        query = "UPDATE `ucs` SET `score` = %s WHERE `uuid` = %s"
        cursor.execute(query, (new_ucs, user_id))
        connection.commit()
    

In [170]:
for user_id in calculated_task_scores['contributor_uuid']:
    ucs_update_score(user_id)

task_scores: 0    0.68241
Name: task_score, dtype: float64
task_scores: 1    0.61278
Name: task_score, dtype: float64
task_scores: 2    0.63
Name: task_score, dtype: float64
task_scores: 3    0.47222
Name: task_score, dtype: float64
task_scores: 4    0.55556
Name: task_score, dtype: float64
task_scores: 5    0.79506
Name: task_score, dtype: float64
task_scores: 6    0.72593
Name: task_score, dtype: float64
task_scores: 7    0.47222
Name: task_score, dtype: float64
task_scores: 8    0.81481
Name: task_score, dtype: float64
task_scores: 9    0.47857
Name: task_score, dtype: float64
task_scores: 10    0.74167
Name: task_score, dtype: float64
task_scores: 11    0.55556
Name: task_score, dtype: float64
task_scores: 12    0.60909
Name: task_score, dtype: float64
task_scores: 13    0.67099
Name: task_score, dtype: float64
task_scores: 14    0.63182
Name: task_score, dtype: float64
task_scores: 15    0.72071
Name: task_score, dtype: float64
task_scores: 16    0.55556
Name: task_score, dtype: f

## end refactor here

#### Step 3) Whitelisting users who are part of the 4/3 demo program

In [36]:
# pseudocode:

# def load_participants_list(file_name):
#     participants = load_data(file_name)
#     combine the first and the second rows of the participants data as the index
#     https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reindex.html
#     drop the second row

# def get_whitelisted_users(participants_list):
#     get ucs scores as df
#     inner merge (?) between ucs scores and participants_list
#     return resulting dataframe, which should already be sorted

#### Step 4) Download the dataframe of whitelisted ucs scores as csv locally

In [37]:
# pseudocode:

# get_whitelisted_users(participants_list)